In [ ]:
!unzip -q /content/Data.zip

replace Imagenes/1682117236778.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
!pip install opencv-python PyYAML ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.2/566.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.3/203.3 kB 26.3 MB/s eta 0:00:00


In [ ]:
import os
from os import path
import xml.etree.ElementTree as ET
import cv2

import numpy as np
import shutil as sh

import matplotlib.pyplot as plt

from utils import pad_image_labels, display_image_and_box_list, display_image_and_box, xmlbox2str, listbox2str, calculate_slice_bboxes
from preprocess_data import *

import albumentations as A

import yaml
from google.colab import files

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

from ultralytics import YOLO

import gc
import torch
import time


Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.4/186.0 GB disk)


In [ ]:
#preprocess_image(raw_images_dir="croppieV2/images", raw_label_dir="croppieV2/labels", output_path="coffee_coco1024", downsize=(768,1024))
adapt_classes(labels_path="labels", classes_json_path="notes.json", convert_all=True, to_ignore=["low_visibility_unsure"])
preprocess_image(raw_images_dir="Imagenes", raw_label_dir="labels", output_path="coffee_coco640", downsize=(480,640))

split_train_val(datadir="coffee_coco640")
#split_train_val(datadir="coffee_coco1024", subname="coffee1024")


{'dark_brown_cherry': '0', 'green_cherry': '1', 'red_cherry': '2', 'yellow_cherry': '3'} ['dark_brown_cherry', 'green_cherry', 'low_visibility_unsure', 'red_cherry', 'yellow_cherry']
No bbox found on  1615496607985.jpg
Problem in bbox value, contains a value bigger than 1
No bbox found on  1615330094166.jpg
No bbox found on  1682104515301.jpg
No bbox found on  1682097742153.jpg
Total images reformat : 4510
Number of images in coffee_coco640 : 4947
Finished Splitting


In [ ]:
wheet_yaml640 = dict(
train ='coffee_coco640/train',
val ='coffee_coco640/val',
nc =1,
names =["cafe verde"]
)

with open('coffee640.yaml', 'w') as outfile:
    yaml.dump(wheet_yaml640, outfile, default_flow_style=True)

In [ ]:
!unzip -q /content/Data.zip

replace Imagenes/1682117236778.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
################# KEEP LOW VISIBILITY / UNSURE ######################

dataset_name = "coffee_coco640_lowviz"

adapt_classes(labels_path="labels", classes_json_path="notes.json", convert_all=True, to_ignore=None)
preprocess_image(raw_images_dir="Imagenes", raw_label_dir="labels", output_path=dataset_name, downsize=(480,640))

split_train_val(datadir=dataset_name)
#split_train_val(datadir="coffee_coco1024", subname="coffee1024")

wheet_yaml640 = dict(
train =f'{dataset_name}/train',
val =f'{dataset_name}/val',
nc =1,
names =["cafe verde"]
)

with open(f'{dataset_name}.yaml', 'w') as outfile:
    yaml.dump(wheet_yaml640, outfile, default_flow_style=True)


No bbox found on  1615496607985.jpg
Problem in bbox value, contains a value bigger than 1
No bbox found on  1615330094166.jpg
No bbox found on  1682104515301.jpg
No bbox found on  1682097742153.jpg
Total images reformat : 4527
Number of images in coffee_coco640_lowviz : 4966
Finished Splitting


In [ ]:
model = YOLO("yolov8s.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coffee_coco640_lowviz.yaml", imgsz=640, batch=10, epochs=60)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
# # results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

New https://pypi.org/project/ultralytics/8.0.98 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=coffee_coco640_lowviz.yaml, epochs=60, patience=50, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=Tru

WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier



60 epochs completed in 4.350 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 22.5MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [01:13<00:00,  1.47s/it]
                   all        994      60440      0.819      0.548      0.618       0.35
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train6


WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier


Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/coffee_coco640_lowviz/val/labels.cache... 994 images, 0 backgrounds, 0 corrupt: 100%|██████████| 994/994 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/datasets/coffee_coco640_lowviz/val/images/1615400721240c9.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640_lowviz/val/images/1615481286338.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640_lowviz/val/images/1615563878875c9.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640_lowviz/val/images/1615565493226c3.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640_lowviz/val/images/1615570409807c12.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640_lowviz/val/images/1615570409807c16.jpg: 1 duplicate lab

WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier


In [ ]:
!unzip -q /content/Data.zip

replace Imagenes/1682117236778.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
################# Multi class detection ######################

dataset_name = "coffee_coco640multi"

adapt_classes(labels_path="labels", classes_json_path="notes.json", convert_all=False, to_ignore=["low_visibility_unsure"])
preprocess_image(raw_images_dir="Imagenes", raw_label_dir="labels", output_path=dataset_name, downsize=(480,640))

split_train_val(datadir=dataset_name)
#split_train_val(datadir="coffee_coco1024", subname="coffee1024")

wheet_yaml640 = dict(
train =f'{dataset_name}/train',
val =f'{dataset_name}/val',
nc=4,
names =["dark_brown", "green_cherry", "red_cherry", "yellow_cherry"]
)

with open(f'{dataset_name}.yaml', 'w') as outfile:
    yaml.dump(wheet_yaml640, outfile, default_flow_style=True)


{'dark_brown_cherry': '0', 'green_cherry': '1', 'red_cherry': '2', 'yellow_cherry': '3'} ['dark_brown_cherry', 'green_cherry', 'low_visibility_unsure', 'red_cherry', 'yellow_cherry']
No bbox found on  1615496607985.jpg
Problem in bbox value, contains a value bigger than 1
No bbox found on  1615330094166.jpg
No bbox found on  1682104515301.jpg
No bbox found on  1682097742153.jpg
Total images reformat : 4510
Number of images in coffee_coco640multi : 4947
Finished Splitting


In [ ]:
model = YOLO("yolov8s.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coffee640.yaml", imgsz=640, batch=10, epochs=80)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
# # results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

del model
del metrics
time.sleep(3)
torch.cuda.empty_cache()
gc.collect()
time.sleep(3)

model = YOLO("yolov8s.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="coffee_coco640multi.yaml", imgsz=640, batch=10, epochs=80)  # train the model
metrics = model.val()  # evaluate model performance on the validation set
# # results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

New https://pypi.org/project/ultralytics/8.0.99 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=coffee640.yaml, epochs=80, patience=50, batch=10, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=to

WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier



80 epochs completed in 5.567 hours.
Optimizer stripped from runs/detect/train11/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 22.5MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [01:08<00:00,  1.36s/it]
                   all        995      58236      0.818      0.564      0.631      0.362
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train11


WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier


Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/coffee_coco640/val/labels.cache... 995 images, 0 backgrounds, 0 corrupt: 100%|██████████| 995/995 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/datasets/coffee_coco640/val/images/1615400721240c9.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640/val/images/1615481286338.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640/val/images/1615563878875c9.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640/val/images/1615565493226c3.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640/val/images/1615570409807c12.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640/val/images/1615570409807c16.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/cof

...

WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier



80 epochs completed in 5.472 hours.
Optimizer stripped from runs/detect/train12/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train12/weights/best.pt, 22.5MB

Validating runs/detect/train12/weights/best.pt...
Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [01:02<00:00,  1.24s/it]
                   all        990      58135      0.548      0.317      0.364      0.194
            dark_brown        990      36675      0.709      0.767      0.771      0.449
          green_cherry        990      21052      0.625      0.277      0.359      0.172
            red_cherry        990        260      0.615      0.203      0.258      0.106
         yellow_cherry        990        148      0.246     0.0203     0.0676     0.0499
Speed: 0.4ms preprocess, 

WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier


Ultralytics YOLOv8.0.96 🚀 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs
val: Scanning /content/datasets/coffee_coco640multi/val/labels.cache... 990 images, 0 backgrounds, 0 corrupt: 100%|██████████| 990/990 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/datasets/coffee_coco640multi/val/images/1615400721240c9.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640multi/val/images/1615481286338.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640multi/val/images/1615563878875c9.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640multi/val/images/1615565493226c3.jpg: 2 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640multi/val/images/1615570409807c12.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/datasets/coffee_coco640multi/val/images/1615570409807c16.jpg: 1 duplicate labels removed
va

WARNING ⚠️ ConfusionMatrix plot failure: Invalid format specifier


In [ ]:
!zip -r /content/results_no_unsure.zip /content/runs/detect/train11/
!zip -r /content/results_multi.zip /content/runs/detect/train12/


  adding: content/runs/detect/train11/ (stored 0%)
  adding: content/runs/detect/train11/train_batch2.jpg (deflated 6%)
  adding: content/runs/detect/train11/val_batch0_pred.jpg (deflated 3%)
  adding: content/runs/detect/train11/R_curve.png (deflated 20%)
  adding: content/runs/detect/train11/confusion_matrix_normalized.png (deflated 36%)
  adding: content/runs/detect/train11/val_batch2_pred.jpg (deflated 3%)
  adding: content/runs/detect/train11/labels_correlogram.jpg (deflated 39%)
  adding: content/runs/detect/train11/train_batch1.jpg (deflated 5%)
  adding: content/runs/detect/train11/labels.jpg (deflated 34%)
  adding: content/runs/detect/train11/results.png (deflated 12%)
  adding: content/runs/detect/train11/val_batch1_labels.jpg (deflated 3%)
  adding: content/runs/detect/train11/results.csv (deflated 85%)
  adding: content/runs/detect/train11/events.out.tfevents.1683924913.9a8e7636985c.827388.1 (deflated 73%)
  adding: content/runs/detect/train11/args.yaml (deflated 51%)
  ad

In [ ]:
files.download("/content/results_multi.zip")
files.download("/content/results_no_unsure.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>